# EDA

In [1]:
%load_ext autoreload
%autoreload 2

from dotenv import dotenv_values
import pandas as pd
config = dotenv_values(".env")
api_key = config["KEY"]

In [2]:
from scrape.tft import TFTScraper

summoner_name = "Oturam"
watcher = TFTScraper(api_key=api_key)

summoner = watcher.from_summoner_name(summoner_name)

In [3]:
match_history = watcher.get_match_history_puuid(summoner.puuid, count=30)
len(match_history)

30

In [14]:
match_details = watcher.get_match_details_from_id(match_history[0])

In [24]:
columns = ["match_id", "data_version", "level", "placement", "puuid"]

name_traits = ["Arcanist", "Assassin", "Bodyguard", "Bruiser", "Challenger", "Colossus",
               "Enchanter", "Innovator", "Protector", "Scholar", "Sniper", "Transformer",
               "Twinshot",]
name_traits = [f"trait_{name}" for name in name_traits]

name_units = [f"unit_{i}" for i in range(9)]
tier_units = [f"tier_unit_{i}" for i in range(9)]

columns.extend(name_traits)
columns.extend(name_units)
columns.extend(tier_units)

df = pd.DataFrame(columns=columns)

In [25]:
from typing import List, Any, Dict

def get_traits(list_traits: List[dict]) -> dict:
    trait = {}
    
    for dic in list_traits:
        name = dic["name"].split("_")[-1]
        trait[name] = dic["tier_total"]
    
    return trait

def get_units(list_units: List[dict]) -> dict:
    units = {}
    
    for dic in list_units:
        name = dic["character_id"].split("_")[-1]
        units[name] = dic["tier"]

    
    return units

def get_participant_data(participant: Dict[Any, Any]) -> Dict[Any, Any]:
    
    participant_dict = {}
    participant_dict["level"] = participant["level"]
    participant_dict["placement"] = participant["placement"]
    participant_dict["puuid"] = participant["puuid"]
    
    return participant_dict

def get_metadata_details(match_details: Dict[Any, Any]) -> Dict[Any, Any]:
    
    try:
        match_dict = match_details.metadata
    
    except:
        raise ValueError("No metadata in match_details")
    
    data_details = {}
    data_details["match_id"] = match_dict["match_id"]
    data_details["data_version"] = match_dict["data_version"]
    return data_details

def get_participant_units(participant: Dict[Any, Any]) -> Dict[Any, Any]:
    
    participant_units = {}
    participant_units["traits"] = get_traits(participant["traits"])
    participant_units["units"] = get_units(participant["units"])
    
    return participant_units

data_details = get_metadata_details(match_details)

columns = df.columns

for player in match_details.info["participants"]:
    
    aux_df = pd.DataFrame(columns=columns, index=[0])
    
    # Participant has the overall data for the player
    participant = get_participant_data(player)
    participant.update(data_details)

    for key, values in participant.items():
        aux_df.loc[0, key] = values


    participant_units = get_participant_units(player)
    
    # Add traits to dataframe
    name_traits = [name for name in columns if "trait" in name.split("_")]
    
    for trait in name_traits:
        if trait in list(participant_units["traits"].keys()):
            aux_df.loc[0, trait] = participant_units["traits"][trait]

    # Add units and tier to dataframe
    name_units = [name for name in columns if "unit" in name.split("_") and len(name.split("_")) == 2]
    tier_units = [name for name in columns if "tier" in name.split("_")]
    
    for index, (name, tier) in enumerate(participant_units["units"].items()):
        aux_df.loc[0, name_units[index]] = name
        aux_df.loc[0, tier_units[index]] = tier
    
    df = df.append(aux_df, ignore_index=True)

df

,match_id,data_version,level,placement,puuid,trait_Arcanist,trait_Assassin,trait_Bodyguard,trait_Bruiser,trait_Challenger,...,unit_8,tier_unit_0,tier_unit_1,tier_unit_2,tier_unit_3,tier_unit_4,tier_unit_5,tier_unit_6,tier_unit_7,tier_unit_8
0,BR1_2445087948,5,8,8,CYWXGzvpi2ggHY6AA7mq_zGwdtsvtKKlkU5PsEUC3reyNB...,NaN,NaN,NaN,NaN,NaN,...,NaN,1,2,2,2,1,1,1,NaN,NaN
1,BR1_2445087948,5,8,7,9sU_lfxN7AvK8tyRKdcKzG5gWzcHz9LHRXbOaTKJZneX-d...,NaN,NaN,NaN,NaN,NaN,...,NaN,2,1,2,1,2,2,1,1,NaN
2,BR1_2445087948,5,8,4,RNAOAfUbuqU9vMHgTkdvuom3IGDSPk8BR1H0jQzX8i4aFx...,NaN,NaN,NaN,NaN,NaN,...,NaN,3,3,3,3,3,2,1,1,NaN
3,BR1_2445087948,5,8,3,raFbU5PamnC1UkFnrzAfdd8OGI9LfK3ERzxAhRdv_EmtgB...,NaN,NaN,NaN,NaN,NaN,...,NaN,3,2,3,2,3,2,1,2,NaN
4,BR1_2445087948,5,8,6,YcQnR0ZG4xHnXP8B9LnwIwH1QYjxiNvrbm3x4b2Os89gzx...,NaN,NaN,NaN,NaN,NaN,...,NaN,2,2,2,3,3,2,1,NaN,NaN
5,BR1_2445087948,5,9,5,5sUMY--M2V_BQAv6dr4kewUTL1OeFUc5BR4NTjpp4r6uCG...,NaN,NaN,NaN,NaN,NaN,...,Jayce,1,1,2,1,2,2,3,2,2
6,BR1_2445087948,5,8,2,AUx12_XDc-eDMs97vZkI8M8y7_WtrLnxL_86yHe8GDKekq...,NaN,NaN,NaN,NaN,NaN,...,NaN,3,3,3,3,3,3,2,2,NaN
7,BR1_2445087948,5,8,1,nic_ehkVUE5owwwsPlwEgXmNej_G8GqIyDqpmzME7-uzlM...,NaN,NaN,NaN,NaN,NaN,...,NaN,2,2,2,2,3,2,2,2,NaN
